In [ ]:
import pandas as pd 

: 

In [ ]:
file_path_stations = r"/Users/melvyn/Documents/maths pour l'info/projet metro/asset/stations.txt"
stations = pd.read_csv(file_path_stations, sep=";", header=0)
stations

In [ ]:
file_path_liaisons = r"/Users/melvyn/Documents/maths pour l'info/projet metro/asset/liaisons.txt"
liaisons = pd.read_csv(file_path_liaisons, sep=" ", header=0)
liaisons

pour chaque sommet : 
    pour chaque autre sommet : 
        verifier si un chemin existe 

parcours en largeur : 
    